In [0]:
import json
import requests

In [0]:
# Azure Blob Storage details
storage_account_name = "<storage-account-name>"
storage_account_key = "<storage-account-access-key>"  
container_name = "input"

In [0]:
# Setting up the Spark configuration for Azure Blob Storage authentication
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

In [0]:
class deltatable_test:
    def __init__(self,delta_table_name):
        self.delta_table_name=delta_table_name
        delta_files_list_dict={}
        for i in dbutils.fs.ls('dbfs:/databricks-datasets/tpch/delta-001/'):
            if i.size ==0:
                delta_files_list_dict[i.name[:-1].capitalize()]=i.path
        self.delta_files_list_dict=delta_files_list_dict
    


    def count_test(self):
        df_source=spark.read.load( self.delta_files_list_dict[self.delta_table_name])
        df_target=spark.read.load("wasbs://output@metadatamanagement02.blob.core.windows.net/replication_folder_delta_tables/"+self.delta_table_name)
        if df_source.count()==df_target.count():
            return True
        else:
            print("There is difference in counts i.e Source Count =" ,df_source.count(),"Target Count is ",df_target.count())
            return False
    def pk_join(self):

        with open('/Workspace/Users/user-email-id/MetaDataManagement_ProjectPro/SourceDefinitionFiles/Delta_Lake/'+self.delta_table_name+'.json', 'r') as f:
                data = json.load(f)
       
        df_source=spark.read.load( self.delta_files_list_dict[self.delta_table_name])
        df_target=spark.read.load("wasbs://output@metadatamanagement02.blob.core.windows.net/replication_folder_delta_tables/"+self.delta_table_name)
        if df_source.join(df_target,data['Primary_key']).count()==df_target.count():
            return True
        else:
            print("There is difference in counts i.e Source Count =" ,df_source.count(),"Target Count is ",df_target.count())
            return False
    


    

        
    

In [0]:
class sql_server:
    def __init__(self,table_name):
        self.table_name=table_name
        server_name = "jdbc:sqlserver://metadatamanagementreplication.database.windows.net"
        database_name = "metadatamanagementreplication"
        username="metadatamanagementserver@metadatamanagementreplication"
        password="<azure-sql-server-password>"
        url = '<azure-sql-server-jdbc-url>'
        connectionProperties={"user":username,"password":password,"driver":"com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        
    #query="SELECT table_name FROM INFORMATION_SCHEMA. TABLES WHERE table_type = 'BASE TABLE' and table_name ="+tab_name+  ";"
        self.url=url
        self.database_name=database_name
        self.server_name=server_name
        self.connectionProperties=connectionProperties
        self.username=username
        self.password=password
        
    


    def count_test(self):
        df_source=spark.read.jdbc(url=self.url,table=self.table_name,properties=self.connectionProperties)
        df_target=spark.read.load("wasbs://output@metadatamanagement02.blob.core.windows.net/sql_server/"+self.table_name)
        if df_source.count()==df_target.count():
            return True
        else:
            print("There is difference in counts i.e Source Count =" ,df_source.count(),"Target Count is ",df_target.count())
            return False
    def pk_join(self):

        with open('/Workspace/Users/user-email-id/MetaDataManagement_ProjectPro/SourceDefinitionFiles/SQL_server/'+self.table_name[4:].capitalize()+'.json', 'r') as f:
                data = json.load(f)
        
        df_source=spark.read.jdbc(url=self.url,table=self.table_name,properties=self.connectionProperties)
        df_target=spark.read.load("wasbs://output@metadatamanagement02.blob.core.windows.net/sql_server/"+self.table_name)
        if df_target.count()==df_source.join(df_target,data['Primary_key']).count():
            return True
        else:
            print("There is difference in counts i.e Source Count =" ,df_source.count(),"Target Count is ",df_target.count())
            return False
    


    

        
    

In [0]:
d1=deltatable_test("Customer")
d1.pk_join();

In [0]:
c1=sql_server("dbo.Employee")

In [0]:
c1.count_test()
c1.pk_join()

Out[11]: True

In [0]:
url = '<logic-app-http-url>'

myobj = {
    "emailaddress": "<email-id>",
    "body": "triggered from Python Databricks",
    "subject": "sample"
}

x = requests.post(url, json = myobj)

print(x.text)